# Setup

In [132]:
import matplotlib.pyplot as plt
from scipy.signal import find_peaks, peak_widths 
import mplcursors
import bisect
from os import mkdir
from shapely.geometry import LineString
%matplotlib qt
import numpy as np

In [133]:
signals = {}
peaks = {}
peaks_w = {}
noise_floor = {}

In [134]:
def extract_data(name_diary):
    multipliers = np.ones((5,3))
    with open(name_diary) as file:
        d = file.readlines()
        i = 0
        j = 0
        while i < len(d):
            if('_ch' in d[i] or 'x:' in d[i]): 
                if 'e' not in d[i+1]:
                    d.insert(i+1, '\n')
                    d.insert(i+1, "1.0e+00 *\n")
                multipliers[(j//3)][j%3] = float(d[i+1][:-3])
                j += 1
            
            i+=1   
        #print(d) 
        #print(multipliers)
        d = "".join(d)
        d = d.split("ch1:")

    d = d[1:]
    for i, s in enumerate(d):
        d[i] = s.split('--')
        d[i] = d[i][0]
        d[i] = d[i].split('\n\n')
        #print(d)
        d[i] = [k for k in d[i] if len(k) > 100]
        d[i] = [a.split('\n') for a in d[i]]
        d[i] = [[n.replace(' ', '').replace('i', 'j') for n in j if len(n.replace(' ', '')) > 0 and 'x' not in n and 'ch' not in n] for j in d[i]]
        #print(len(d[i][0]))
        d[i] = [multipliers[i][k]*np.array([complex(n) if k < 2 else float(n) for n in j])[:1000] for k, j in enumerate(d[i])]
    return d

In [135]:
def get_widths(x, y, peak_i):
    contour = y[peak_i]*0.5
    data = LineString(np.column_stack((x, y)))
    target = LineString(((x[0], contour), (x[-1], contour)))
    intercepts = data.intersection(target)
    cepts = []
    if intercepts.is_empty:
        return np.array([0, x[peak_i], x[peak_i]])
    
    if intercepts.geom_type == "Point":
        cepts = np.array([intercepts.x])
    else:
        cepts = np.array([i.x for i in intercepts.geoms])
    comps = np.sort(cepts-x[peak_i])
    #print(comps)
    right = bisect.bisect_right(comps,0)
    return np.array([contour, comps[right-1]+x[peak_i], comps[min(right, len(comps) - 1)]+x[peak_i]])

In [136]:
def plot_data(key, typ, run, rwidths_plots=[], width = False, combine = False):
    d = signals[key]
    if typ == "transfer":
        data = np.abs(d[run][1])/np.abs(d[run][0])
    elif typ == "out":
        data = np.abs(d[run][1])
    else:
        data = np.abs(d[run][0])
    try:
        mkdir(f"{key}")
    except:
        pass
    if(typ == "transfer"):
        plt.plot(d[run][2], 20*np.log10(data), label=key, lw=0.75)
        plt.ylabel("Intensity (dB)")
    else:
        plt.plot(d[run][2], data, label=key, lw=0.75)
        plt.ylabel("Intensity (arbitrary units)")
    plt.xlabel("Frequency (Hz)")
    if(width):
        plt.hlines(*rwidths_plots, lw=3, label="width", color="C1")
    plt.title(f"{typ}, run={run}")
    '''
    Run 0 = sweep
    Run 1 = sine at 9.2Hz
    Run 2 = random
    Run 3 = sines at 9.2 and 13.6
    Run 4 = sines at 9.2 and 50
    '''
    plt.legend()
    #mplcursors.cursor(hover=True)
    plt.show()
    if not combine:
        plt.savefig(f"{key}/{typ}-{run}.pdf")
        #plt.pause(5)
        plt.close()
    else:
        mplcursors.cursor(hover=True)

In [137]:
def get_data(key,typ="transfer", run=0, plot=False, combine=False):
    '''
    Returns peaks, widths (6dB), plottable widths, and noise floor
    '''
    d = signals[key]
    if typ == "transfer":
        data = np.abs(d[run][1])/np.abs(d[run][0])
    elif typ == "out":
        data = np.abs(d[run][1])
    else:
        data = np.abs(d[run][0])
    prominences = np.linspace(0, 10000, int(1e8))[::-1]
    precise_pr = bisect.bisect_left(prominences, 3, key=lambda x: len(find_peaks(data, prominence=x,distance=3)[0]))
    #print(precise_height)
    rpeaks = find_peaks(data, prominence=prominences[precise_pr],distance=30)[0]
    #print(get_widths(d[0][2], data, rpeaks[0]))
    #temp = peak_widths(data, rpeaks, rel_height=0.5) 
    #rwidths, rwidths_plots = temp[0], temp[1:]
    #rwidths_plots = [20*np.log10(x) if i == 0 else x/10 for i,x in enumerate(rwidths_plots)]
    rwidths_plots = list(zip(*[get_widths(d[run][2], data, p) for p in rpeaks]))
    #print(rwidths_plots)
    #print(peak_widths(data, rpeaks, rel_height=0.5)[1:])
    if typ == "transfer":
        rwidths_plots = [20*np.log10(np.array(x)) if i == 0 else np.array(x) for i,x in enumerate(rwidths_plots)]
    else:
        rwidths_plots = [np.array(x) if i == 0 else np.array(x) for i,x in enumerate(rwidths_plots)]
    #print(rwidths_plots)
    rwidths = [rwidths_plots[2][i] -rwidths_plots[1][i] for i in range(len(rwidths_plots[0]))]
    if typ != "transfer":
        noise = np.average(data[400:600])
    else:
        noise = np.average(np.nan_to_num(np.abs(20*np.log10(data[400:600])), posinf=0, nan=0))
    if(plot):
        plot_data(key, typ=typ, run=run, rwidths_plots=rwidths_plots, width=True,combine=False)
    list_of_peaks = list(d[run][2][rpeaks])
    peak_heigths = list(data[rpeaks])
    if list_of_peaks[0] > 6:
        list_of_peaks.insert(0,0)
        peak_heigths.insert(0,0)
    if list_of_peaks[-1] > 15:
        list_of_peaks = list_of_peaks[:-1]
        peak_heigths = peak_heigths[:-1]
    if list_of_peaks[-1] < 11:
        list_of_peaks.append(0)
        peak_heigths.append(0)
    if len(peak_heigths) < 3:
        list_of_peaks.insert(1,0)
        peak_heigths.insert(1,0)
    if len(peak_heigths) != 3:
        print(d[run][2][rpeaks], data[rpeaks])
        raise Exception('AAAAAAAAAAA')
    return (list_of_peaks, peak_heigths), rwidths, rwidths_plots, noise

# Individual signals

## Kaiser

### Kaiser 1

In [138]:
key = "Kaiser (param 1)"
signals[key] = extract_data("code/myKaiser1Diary")
for k in [peaks, peaks_w, noise_floor]:
    k[key] = {}
for L in ["transfer", "out", "in"]:
    for k in [peaks, peaks_w, noise_floor]:
        k[key][L] = [0,0,0,0,0]
    for r in range(0,5):
        my_data = get_data(key, typ = L, run = r,plot=True)
        
        peaks[key][L][r] = my_data[0]
        peaks_w[key][L][r] = my_data[1:3]
        noise_floor[key][L][r] = my_data[3]



C:\Users\gabri\AppData\Local\Temp\ipykernel_24736\1848065966.py:32: RuntimeWarning: divide by zero encountered in log10
  noise = np.average(np.nan_to_num(np.abs(20*np.log10(data[400:600])), posinf=0, nan=0))
C:\Users\gabri\AppData\Local\Temp\ipykernel_24736\2921350288.py:14: RuntimeWarning: divide by zero encountered in log10
  plt.plot(d[run][2], 20*np.log10(data), label=key, lw=0.75)
C:\Users\gabri\AppData\Local\Temp\ipykernel_24736\2921350288.py:14: RuntimeWarning: divide by zero encountered in log10
  plt.plot(d[run][2], 20*np.log10(data), label=key, lw=0.75)
C:\Users\gabri\AppData\Local\Temp\ipykernel_24736\1848065966.py:32: RuntimeWarning: divide by zero encountered in log10
  noise = np.average(np.nan_to_num(np.abs(20*np.log10(data[400:600])), posinf=0, nan=0))
C:\Users\gabri\AppData\Local\Temp\ipykernel_24736\2921350288.py:14: RuntimeWarning: divide by zero encountered in log10
  plt.plot(d[run][2], 20*np.log10(data), label=key, lw=0.75)


### Kaiser 3

In [139]:
key = "Kaiser (param 3)"
signals[key] = extract_data("code/myKaiser3Diary")
for k in [peaks, peaks_w, noise_floor]:
    k[key] = {}
for L in ["transfer", "out", "in"]:
    for k in [peaks, peaks_w, noise_floor]:
        k[key][L] = [0,0,0,0,0]
    for r in range(0,5):
        my_data = get_data(key, typ = L, run = r,plot=True)
        
        peaks[key][L][r] = my_data[0]
        peaks_w[key][L][r] = my_data[1:3]
        noise_floor[key][L][r] = my_data[3]

C:\Users\gabri\AppData\Local\Temp\ipykernel_24736\2921350288.py:14: RuntimeWarning: divide by zero encountered in log10
  plt.plot(d[run][2], 20*np.log10(data), label=key, lw=0.75)
C:\Users\gabri\AppData\Local\Temp\ipykernel_24736\2921350288.py:14: RuntimeWarning: divide by zero encountered in log10
  plt.plot(d[run][2], 20*np.log10(data), label=key, lw=0.75)


### Kaiser 9

In [140]:
key = "Kaiser (param 9)"
signals[key] = extract_data("code/myKaiser9Diary")
for k in [peaks, peaks_w, noise_floor]:
    k[key] = {}
for L in ["transfer", "out", "in"]:
    for k in [peaks, peaks_w, noise_floor]:
        k[key][L] = [0,0,0,0,0]
    for r in range(0,5):
        my_data = get_data(key, typ = L, run = r,plot=True)
        
        peaks[key][L][r] = my_data[0]
        peaks_w[key][L][r] = my_data[1:3]
        noise_floor[key][L][r] = my_data[3]

C:\Users\gabri\AppData\Local\Temp\ipykernel_24736\1848065966.py:7: RuntimeWarning: divide by zero encountered in divide
  data = np.abs(d[run][1])/np.abs(d[run][0])
c:\Users\gabri\mystuff\cambridgeStuff\a4-extended\.venv\Lib\site-packages\shapely\set_operations.py:168: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
C:\Users\gabri\AppData\Local\Temp\ipykernel_24736\1848065966.py:24: RuntimeWarning: divide by zero encountered in log10
  rwidths_plots = [20*np.log10(np.array(x)) if i == 0 else np.array(x) for i,x in enumerate(rwidths_plots)]
C:\Users\gabri\AppData\Local\Temp\ipykernel_24736\1848065966.py:32: RuntimeWarning: divide by zero encountered in log10
  noise = np.average(np.nan_to_num(np.abs(20*np.log10(data[400:600])), posinf=0, nan=0))
C:\Users\gabri\AppData\Local\Temp\ipykernel_24736\2921350288.py:4: RuntimeWarning: divide by zero encountered in divide
  data = np.abs(d[run][1])/np.abs(d[run][0])
C:\Users\gabri\AppData\Local

## Flat-top

In [141]:
key = "Flat-top"
signals[key] = extract_data("code/myFlattopDiary")
for k in [peaks, peaks_w, noise_floor]:
    k[key] = {}
for L in ["transfer", "out", "in"]:
    for k in [peaks, peaks_w, noise_floor]:
        k[key][L] = [0,0,0,0,0]
    for r in range(0,5):
        my_data = get_data(key, typ = L, run = r,plot=True)
        
        peaks[key][L][r] = my_data[0]
        peaks_w[key][L][r] = my_data[1:3]
        noise_floor[key][L][r] = my_data[3]

C:\Users\gabri\AppData\Local\Temp\ipykernel_24736\2921350288.py:14: RuntimeWarning: divide by zero encountered in log10
  plt.plot(d[run][2], 20*np.log10(data), label=key, lw=0.75)
C:\Users\gabri\AppData\Local\Temp\ipykernel_24736\1848065966.py:7: RuntimeWarning: divide by zero encountered in divide
  data = np.abs(d[run][1])/np.abs(d[run][0])
c:\Users\gabri\mystuff\cambridgeStuff\a4-extended\.venv\Lib\site-packages\shapely\set_operations.py:168: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
C:\Users\gabri\AppData\Local\Temp\ipykernel_24736\1848065966.py:24: RuntimeWarning: divide by zero encountered in log10
  rwidths_plots = [20*np.log10(np.array(x)) if i == 0 else np.array(x) for i,x in enumerate(rwidths_plots)]
C:\Users\gabri\AppData\Local\Temp\ipykernel_24736\2921350288.py:4: RuntimeWarning: divide by zero encountered in divide
  data = np.abs(d[run][1])/np.abs(d[run][0])
C:\Users\gabri\AppData\Local\Temp\ipykernel_24736\292135

## Blackman

In [142]:
key = "Blackman"
signals[key] = extract_data("code/myBlackmanDiary")
for k in [peaks, peaks_w, noise_floor]:
    k[key] = {}
for L in ["transfer", "out", "in"]:
    for k in [peaks, peaks_w, noise_floor]:
        k[key][L] = [0,0,0,0,0]
    for r in range(0,5):
        my_data = get_data(key, typ = L, run = r,plot=True)
        
        peaks[key][L][r] = my_data[0]
        peaks_w[key][L][r] = my_data[1:3]
        noise_floor[key][L][r] = my_data[3]

C:\Users\gabri\AppData\Local\Temp\ipykernel_24736\1848065966.py:7: RuntimeWarning: divide by zero encountered in divide
  data = np.abs(d[run][1])/np.abs(d[run][0])
c:\Users\gabri\mystuff\cambridgeStuff\a4-extended\.venv\Lib\site-packages\shapely\set_operations.py:168: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
C:\Users\gabri\AppData\Local\Temp\ipykernel_24736\1848065966.py:24: RuntimeWarning: divide by zero encountered in log10
  rwidths_plots = [20*np.log10(np.array(x)) if i == 0 else np.array(x) for i,x in enumerate(rwidths_plots)]
C:\Users\gabri\AppData\Local\Temp\ipykernel_24736\1848065966.py:32: RuntimeWarning: divide by zero encountered in log10
  noise = np.average(np.nan_to_num(np.abs(20*np.log10(data[400:600])), posinf=0, nan=0))
C:\Users\gabri\AppData\Local\Temp\ipykernel_24736\2921350288.py:4: RuntimeWarning: divide by zero encountered in divide
  data = np.abs(d[run][1])/np.abs(d[run][0])
C:\Users\gabri\AppData\Local

## Rect (10s)

In [143]:
key = "No windowing"
signals[key] = extract_data("code/myRectDiary")
for k in [peaks, peaks_w, noise_floor]:
    k[key] = {}
for L in ["transfer", "out", "in"]:
    for k in [peaks, peaks_w, noise_floor]:
        k[key][L] = [0,0,0,0,0]
    for r in range(0,5):
        my_data = get_data(key, typ = L, run = r,plot=True)
        
        peaks[key][L][r] = my_data[0]
        peaks_w[key][L][r] = my_data[1:3]
        noise_floor[key][L][r] = my_data[3]

C:\Users\gabri\AppData\Local\Temp\ipykernel_24736\1848065966.py:7: RuntimeWarning: invalid value encountered in divide
  data = np.abs(d[run][1])/np.abs(d[run][0])
c:\Users\gabri\mystuff\cambridgeStuff\a4-extended\.venv\Lib\site-packages\shapely\creation.py:218: RuntimeWarning: invalid value encountered in linestrings
  return lib.linestrings(coords, np.intc(handle_nan), out=out, **kwargs)
c:\Users\gabri\mystuff\cambridgeStuff\a4-extended\.venv\Lib\site-packages\shapely\set_operations.py:168: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
C:\Users\gabri\AppData\Local\Temp\ipykernel_24736\2921350288.py:4: RuntimeWarning: invalid value encountered in divide
  data = np.abs(d[run][1])/np.abs(d[run][0])
C:\Users\gabri\AppData\Local\Temp\ipykernel_24736\1848065966.py:7: RuntimeWarning: invalid value encountered in divide
  data = np.abs(d[run][1])/np.abs(d[run][0])
c:\Users\gabri\mystuff\cambridgeStuff\a4-extended\.venv\Lib\site-packages\

## Chebyshev

### Chebyshev 20

In [144]:
key = "Chebyshev (-20dB)"
signals[key] = extract_data("code/myCheb20Diary")
for k in [peaks, peaks_w, noise_floor]:
    k[key] = {}
for L in ["transfer", "out", "in"]:
    for k in [peaks, peaks_w, noise_floor]:
        k[key][L] = [0,0,0,0,0]
    for r in range(0,5):
        my_data = get_data(key, typ = L, run = r,plot=True)
        
        peaks[key][L][r] = my_data[0]
        peaks_w[key][L][r] = my_data[1:3]
        noise_floor[key][L][r] = my_data[3]

C:\Users\gabri\AppData\Local\Temp\ipykernel_24736\1848065966.py:24: RuntimeWarning: divide by zero encountered in log10
  rwidths_plots = [20*np.log10(np.array(x)) if i == 0 else np.array(x) for i,x in enumerate(rwidths_plots)]
C:\Users\gabri\AppData\Local\Temp\ipykernel_24736\1848065966.py:24: RuntimeWarning: divide by zero encountered in log10
  rwidths_plots = [20*np.log10(np.array(x)) if i == 0 else np.array(x) for i,x in enumerate(rwidths_plots)]


### Chebyshev 60

In [145]:
key = "Chebyshev (-60dB)"
signals[key] = extract_data("code/myCheb60Diary")
for k in [peaks, peaks_w, noise_floor]:
    k[key] = {}
for L in ["transfer", "out", "in"]:
    for k in [peaks, peaks_w, noise_floor]:
        k[key][L] = [0,0,0,0,0]
    for r in range(0,5):
        my_data = get_data(key, typ = L, run = r,plot=True)
        
        peaks[key][L][r] = my_data[0]
        peaks_w[key][L][r] = my_data[1:3]
        noise_floor[key][L][r] = my_data[3]

### Chebyshev 100

In [146]:
key = "Chebyshev (-100dB)"
signals[key] = extract_data("code/myCheb100Diary")
for k in [peaks, peaks_w, noise_floor]:
    k[key] = {}
for L in ["transfer", "out", "in"]:
    for k in [peaks, peaks_w, noise_floor]:
        k[key][L] = [0,0,0,0,0]
    for r in range(0,5):
        my_data = get_data(key, typ = L, run = r,plot=True)
        
        peaks[key][L][r] = my_data[0]
        peaks_w[key][L][r] = my_data[1:3]
        noise_floor[key][L][r] = my_data[3]

C:\Users\gabri\AppData\Local\Temp\ipykernel_24736\1848065966.py:32: RuntimeWarning: divide by zero encountered in log10
  noise = np.average(np.nan_to_num(np.abs(20*np.log10(data[400:600])), posinf=0, nan=0))
C:\Users\gabri\AppData\Local\Temp\ipykernel_24736\2921350288.py:14: RuntimeWarning: divide by zero encountered in log10
  plt.plot(d[run][2], 20*np.log10(data), label=key, lw=0.75)
C:\Users\gabri\AppData\Local\Temp\ipykernel_24736\1848065966.py:32: RuntimeWarning: divide by zero encountered in log10
  noise = np.average(np.nan_to_num(np.abs(20*np.log10(data[400:600])), posinf=0, nan=0))
C:\Users\gabri\AppData\Local\Temp\ipykernel_24736\2921350288.py:14: RuntimeWarning: divide by zero encountered in log10
  plt.plot(d[run][2], 20*np.log10(data), label=key, lw=0.75)
C:\Users\gabri\AppData\Local\Temp\ipykernel_24736\1848065966.py:7: RuntimeWarning: divide by zero encountered in divide
  data = np.abs(d[run][1])/np.abs(d[run][0])
c:\Users\gabri\mystuff\cambridgeStuff\a4-extended\.venv

# Combined graphs

## Run 0 (sweep)

### Transfer function

In [90]:
for g in ["transfer"]:
    for sig in signals.keys():
        #print(sig)
        plot_data(sig, typ=g, run=0, combine=True, width=False)
    plt.savefig(f"Run 0 -- combined -- {g}.pdf")
    plt.close()

C:\Users\gabri\AppData\Local\Temp\ipykernel_12156\2921350288.py:4: RuntimeWarning: invalid value encountered in divide
  data = np.abs(d[run][1])/np.abs(d[run][0])


In [136]:
colourmap = plt.get_cmap("turbo")
data = np.array([noise_floor[s]["transfer"][0] for s in signals.keys()])
plt.bar(signals.keys(), data, color=colourmap((data-np.min(data))/(np.max(data)-np.min(data))))
plt.xticks(rotation=90)
plt.ylabel("Noise floor (dB)")
plt.title("Noise floor of output signal at sweep")
plt.savefig("noise_floor_transfer_run_0_out.pdf",bbox_inches='tight')

In [154]:
###WIDTHS
colourmap = plt.get_cmap("turbo")
data = [peaks_w[s]["transfer"][0][0] for s in signals.keys()]
max_L = 0
print(data)

x_poses = np.arange(len(data))
w = 0.3
aa = ["Peak 1","Peak 2","Peak 3"]
data = list(zip(*data))
print(data)
for i, peeks in enumerate(data[:3]):
    plt.bar(x_poses + i*w - w/2, peeks, width=w,label=f"Peak {i}")

plt.xticks(x_poses, signals.keys(), rotation=90)
plt.legend()
plt.ylabel("Peak width (Hz)")
plt.title("Peak width of transfer at sweep input")
plt.savefig("peak_width_transfer_run_0_out.pdf",bbox_inches='tight')

[[np.float64(0.1757277256688612), np.float64(0.13874933108493614), np.float64(0.19051854173343763)], [np.float64(0.11044996397779538), np.float64(0.33604796286074645), np.float64(0.19177973369960277)], [np.float64(0.12171162418574788), np.float64(0.6497019170745872), 0], [np.float64(0.1056010318715046), 0, 0], [np.float64(0.1259250846158576), np.float64(0.11111824948750026), np.float64(0.6354929064367774)], [np.float64(0.1557188739355806), np.float64(0.12485075933361323), np.float64(0.2121497017827494)], [np.float64(0.2348795107588515), np.float64(0.18508349283525938), 0], [np.float64(0.1498948506882306), np.float64(0.12320835264035779), np.float64(0.318709545187863)], [np.float64(0.12545103482858888), np.float64(0.11644958002629835), 0]]
[(np.float64(0.1757277256688612), np.float64(0.11044996397779538), np.float64(0.12171162418574788), np.float64(0.1056010318715046), np.float64(0.1259250846158576), np.float64(0.1557188739355806), np.float64(0.2348795107588515), np.float64(0.1498948506

In [155]:
###FREQS
colourmap = plt.get_cmap("turbo")
data = [list(peaks[s]["transfer"][0][0]) for s in signals.keys()][:-1]
max_L = 0
print(data)

x_poses = np.arange(len(data[0]))
w = 0.08
aa = ["Peak 1","Peak 2","Peak 3"]
#data = list(zip(*data))
print(data)
for i, peeks in enumerate(data):
    plt.bar(x_poses + i*w - 3.5*w, peeks, width=w,label=f"{list(signals.keys())[:-1][i]}")

plt.xticks(x_poses, aa, rotation=90)
plt.legend()
plt.ylabel("Peak place (Hz)")
plt.title("Largest 3 peaks of transfer at sweep input")
plt.savefig("peak_places_transfer_run_0_out.pdf",bbox_inches='tight')

[[np.float64(3.2011), np.float64(9.2032), np.float64(13.5046)], [np.float64(0.8003), np.float64(9.2032), np.float64(13.5046)], [0, np.float64(7.8027), np.float64(13.5046)], [0, np.float64(9.0031), 0], [np.float64(0.3001), np.float64(8.803), np.float64(13.5046)], [np.float64(3.2011), np.float64(9.2032), np.float64(13.5046)], [0, np.float64(10.1035), np.float64(13.8047)], [np.float64(1.0003), np.float64(8.703), np.float64(13.4046)]]
[[np.float64(3.2011), np.float64(9.2032), np.float64(13.5046)], [np.float64(0.8003), np.float64(9.2032), np.float64(13.5046)], [0, np.float64(7.8027), np.float64(13.5046)], [0, np.float64(9.0031), 0], [np.float64(0.3001), np.float64(8.803), np.float64(13.5046)], [np.float64(3.2011), np.float64(9.2032), np.float64(13.5046)], [0, np.float64(10.1035), np.float64(13.8047)], [np.float64(1.0003), np.float64(8.703), np.float64(13.4046)]]


### Separate channels

#### Channel 1 (input force)

In [92]:
for g in ["out"]:
    for sig in signals.keys():
        #print(sig)
        plot_data(sig, typ=g, run=0, combine=True, width=False)
    plt.savefig(f"Run 0 -- combined -- {g}.pdf")
    plt.close()

#### Channel 2 (output)

In [91]:
for g in ["in"]:
    for sig in signals.keys():
        #print(sig)
        plot_data(sig, typ=g, run=0, combine=True, width=False)
    plt.savefig(f"Run 0 -- combined -- {g}.pdf")
    plt.close()

## Run 1 (pure resonance at 9.2 Hz)

In [119]:
for g in ["out","in"][::-1]:
    for sig in signals.keys():
        #print(sig)
        plot_data(sig, typ=g, run=1, combine=True, width=False)
    plt.savefig(f"Run 1 -- combined -- {g}.pdf")
    plt.close()

In [137]:
colourmap = plt.get_cmap("turbo")
data = np.array([noise_floor[s]["out"][1] for s in signals.keys()])
plt.bar(signals.keys(), data, color=colourmap(data*2/np.max(data)))
plt.xticks(rotation=90)
plt.ylabel("Noise floor (arbitrary units)")
plt.title("Noise floor of output signal at 9.2 Hz input")
plt.savefig("noise_floor_run_1_out.pdf",bbox_inches='tight')

In [152]:
colourmap = plt.get_cmap("turbo")
data = [peaks_w[s]["out"][1][0] for s in signals.keys()]
max_L = 0
print(data)

x_poses = np.arange(len(data))
w = 0.3
aa = ["Peak 1","Peak 2","Peak 3"]
data = list(zip(*data))
print(data)
for i, peeks in enumerate(data[:3]):
    plt.bar(x_poses + i*w - w/2, peeks, width=w,label=f"Peak {i}")

plt.xticks(x_poses, signals.keys(), rotation=90)
plt.legend()
plt.ylabel("Peak width (Hz)")
plt.title("Peak width of output signal at 9.2 Hz input")
plt.savefig("peak_width_run_1_out.pdf",bbox_inches='tight')

[[np.float64(0.10493565859396625), np.float64(0.10688930596254664), np.float64(0.1365717327486422)], [np.float64(0.13484189824525572), np.float64(0.13453196075718665), np.float64(0.16908821399625396)], [np.float64(0.24276817534180495), np.float64(0.2617634451373938), np.float64(0.2391508647761178)], [np.float64(0.46407058800627254), np.float64(0.4664966833051558), np.float64(0.5333922057761242)], [np.float64(0.2382211279806583), np.float64(0.23790369208759898), np.float64(0.2243737766206877)], [np.float64(0.10015546525065133), np.float64(0.10145179070246613), np.float64(0.1844270164185957)], [np.float64(0.11081266693299519), 0, 0], [np.float64(0.20284636929429567), np.float64(0.22883742057367584), np.float64(0.16718022150801914)], [np.float64(0.2669922345334861), np.float64(0.2617707914590568), np.float64(0.27936937128459505)]]
[(np.float64(0.10493565859396625), np.float64(0.13484189824525572), np.float64(0.24276817534180495), np.float64(0.46407058800627254), np.float64(0.2382211279806

In [18]:
colourmap = plt.get_cmap("turbo")

data = np.array([peaks[s]["out"][1][1][0] for s in signals.keys()])

plt.bar(signals.keys(), data, color=colourmap(data/np.max(data)))
plt.xticks(rotation=90)
plt.ylabel("Main peak amplitude (arbitrary units)")
plt.title("Main peak amplitude of output signal at 9.2 Hz input")
plt.savefig("main_peak_amplitude_run_1_out.pdf",bbox_inches='tight')

## Run 2 (random)

In [48]:
for g in ["transfer"]:
    for sig in signals.keys():
        #print(sig)
        plot_data(sig, typ=g, run=2, combine=True, width=False)
    plt.savefig(f"Run 2 -- combined -- {g}.pdf")
    plt.close()

C:\Users\gabri\AppData\Local\Temp\ipykernel_12156\2921350288.py:4: RuntimeWarning: invalid value encountered in divide
  data = np.abs(d[run][1])/np.abs(d[run][0])


In [138]:
colourmap = plt.get_cmap("turbo")
data = np.array([noise_floor[s]["transfer"][2] for s in signals.keys()])
plt.bar(signals.keys(), data, color=colourmap((data-np.min(data))/(np.max(data)-np.min(data))))
plt.xticks(rotation=90)
plt.ylabel("Intensity (f_ch2/f_ch1)")
plt.title("Transfer function noise floor inferred from random input")
plt.savefig("noise_floor_run_2_out.pdf",bbox_inches='tight')